# Informe de Prácticas: Técnicas de Algoritmos de Búsqueda

## Introducción

Para esta asignatura, he desarrollado una serie de prácticas enfocadas en técnicas de algoritmos de búsqueda aplicadas al problema de correspondencia de grafos (graph matching). El trabajo se ha centrado principalmente en la visualización, construcción y matching de grafos basados en puntos clave (keypoints) de imágenes del conjunto de datos Willow-Object-Class.

Este conjunto de datos contiene imágenes de cinco categorías (coche, cara, pato, motocicleta y botella de vino), cada una anotada con keypoints específicos. El objetivo principal ha sido implementar y comparar diferentes algoritmos para establecer correspondencias entre puntos característicos de pares de imágenes, evaluando su precisión y rendimiento.

He estructurado el trabajo en varios módulos que describiré a continuación, comenzando por la infraestructura básica y avanzando hacia técnicas más sofisticadas de correspondencia de grafos.

## DataLoader

Para facilitar el trabajo con los datos, decidí crear una clase externa llamada `DataLoader` que se encarga de leer y organizar todas las carpetas de datos. Esta clase escanea el directorio base (por defecto, la carpeta 'data' del proyecto) y busca imágenes `.png` junto con sus correspondientes archivos `.mat` de keypoints.

La clase almacena las rutas en dataframes de pandas organizados por categorías, permitiendo un acceso rápido y estructurado a los datos desde cualquier notebook. Entre sus funcionalidades destacan:

- Carga automática de pares de archivos (imagen + keypoints) organizados por categorías
- Generación de dataframes individuales para cada categoría
- Posibilidad de obtener un dataframe combinado de todas las categorías
- Método para seleccionar imágenes aleatorias de una categoría específica

Esta herramienta resultó fundamental para mantener el código limpio y modular a lo largo de todas las prácticas, evitando duplicación de código y proporcionando una interfaz consistente para acceder a los datos.

## Visualización de Imágenes y Keypoints - Parte 1

En esta primera parte del trabajo, me centré en la visualización básica de las imágenes y sus puntos característicos. Desarrollé funciones para:

1. Configurar correctamente el PYTHONPATH para poder importar mi módulo DataLoader
2. Cargar imágenes y sus correspondientes keypoints
3. Redimensionar todas las imágenes a un tamaño uniforme de 256x256 píxeles
4. Ajustar las coordenadas de los keypoints para mantener su alineación con las imágenes redimensionadas
5. Visualizar conjuntos de imágenes en grids de 2x4 con sus keypoints superpuestos

Para cada categoría (coche, cara, pato, motocicleta y botella de vino), seleccioné aleatoriamente 8 imágenes y las visualicé con sus puntos característicos. Este ejercicio me permitió familiarizarme con el dataset y verificar que la carga de datos funcionaba correctamente.

La función `load_img_and_keypoints_from_row` resultó especialmente útil, ya que abstrae todo el proceso de carga y redimensionamiento, permitiéndome reutilizarla en las siguientes partes de la práctica.

## Visualización de Imágenes y Keypoints - Parte 2

En la segunda parte de la visualización, avancé hacia la construcción de grafos basados en los keypoints. Implementé dos métodos principales:

### Triangulación de Delaunay

La triangulación de Delaunay es una técnica geométrica que conecta puntos formando triángulos que maximizan el ángulo mínimo, evitando triángulos muy delgados. El algoritmo que implementé:

1. Obtiene los keypoints de una imagen
2. Verifica si los puntos forman triángulos mediante la comprobación de si están dentro del círculo circunscrito
3. Genera un conjunto de aristas que conectan los puntos según la triangulación
4. Visualiza estas aristas superpuestas sobre las imágenes originales

Este método proporciona una forma robusta de conectar los keypoints basándose únicamente en su distribución espacial, sin necesidad de parámetros adicionales.

### Grafos K-NN (K vecinos más cercanos)

Como alternativa a Delaunay, implementé también la construcción de grafos K-NN, donde cada punto se conecta con sus K vecinos más cercanos. Este método:

1. Calcula todas las distancias euclidianas entre pares de puntos
2. Para cada punto, identifica sus K vecinos más cercanos
3. Establece conexiones entre el punto y sus vecinos
4. Visualiza estas conexiones sobre las imágenes

Experimenté con diferentes valores de K (3, 5 y 7) para todas las categorías de imágenes, lo que permitió observar cómo la densidad de conexiones afecta a la estructura del grafo resultante.

La visualización de ambos métodos me permitió compararlos y entender sus diferencias: mientras que Delaunay genera una triangulación uniforme, K-NN tiende a crear más conexiones en áreas donde los puntos están más concentrados.

## Generación de Grafos Matcheados - Parte 1

En esta etapa, avancé hacia la correspondencia entre pares de imágenes utilizando los grafos construidos anteriormente. El objetivo era encontrar qué keypoints de una imagen se corresponden con los de otra imagen de la misma categoría.

Implementé una función `visualize_combined` que muestra dos imágenes lado a lado con sus grafos y las correspondencias entre puntos. Para establecer estas correspondencias, desarrollé:

1. Una función de carga y preprocesamiento de pares de imágenes
2. Un método para construir matrices de adyacencia basadas en triangulación de Delaunay
3. Un algoritmo de matching espacial que:
   - Calcula una matriz de costes basada en distancias euclidianas entre todos los pares de puntos
   - Utiliza el algoritmo húngaro (linear_sum_assignment de scipy) para encontrar la asignación óptima
   - Genera una matriz binaria de correspondencias

Las correspondencias correctas se visualizan con líneas verdes y las incorrectas con líneas rojas, permitiendo evaluar visualmente la calidad del matching. Este enfoque básico, basado únicamente en coordenadas espaciales, funciona razonablemente bien para imágenes similares pero tiene limitaciones cuando las posiciones de los objetos varían significativamente.

## Generación de Grafos Matcheados - Parte 2

En la parte final y más avanzada, mejoré el algoritmo de matching incorporando características topológicas además de las espaciales. Las principales innovaciones fueron:

1. **Implementación de Node2Vec**: Esta técnica genera embeddings vectoriales para cada nodo del grafo que capturan información sobre su contexto estructural y su posición en la red. Los nodos con roles similares en distintos grafos tendrán embeddings similares.

2. **Tiempos de llegada (Hitting Times)**: Calculé matrices que representan el número esperado de pasos que tarda un paseo aleatorio en llegar de un nodo a otro, capturando información global sobre la conectividad del grafo.

3. **Matching mejorado**: Combiné las características espaciales con las topológicas para crear una matriz de costes ponderada, balanceando la importancia de la posición espacial y la estructura del grafo.

Además, implementé un sistema de evaluación automática que calcula la precisión de los matchings para todas las categorías de imágenes, generando estadísticas como la precisión media y la desviación estándar para cada categoría.

También experimenté con distintas configuraciones de pesos para las diferentes características, analizando cómo afectaban a la precisión final del matching.

## Evaluación de Resultados

Según los resultados obtenidos y guardados en los archivos CSV, puedo concluir que:

1. La incorporación de características topológicas mejoró significativamente la precisión del matching en todas las categorías, como se puede observar comparando results.csv (enfoque espacial básico) con results2.csv (enfoque mejorado).

2. Las categorías "face" y "winebottle" obtuvieron los mejores resultados de matching, con precisiones del 90.92% y 89.31% respectivamente en el enfoque mejorado.

3. La categoría "duck" presentó la menor precisión (70.72%), probablemente debido a que los keypoints son más ambiguos o la variabilidad entre imágenes es mayor.

4. El método mejorado no sólo incrementó la precisión media, sino que también redujo la desviación estándar en todas las categorías, lo que indica un comportamiento más consistente.

En conclusión, este trabajo me ha permitido implementar y comprender diversas técnicas de correspondencia de grafos, desde los enfoques más básicos hasta métodos avanzados que incorporan información topológica. Los resultados demuestran claramente que considerar la estructura del grafo, además de la posición espacial de los keypoints, mejora significativamente la capacidad de establecer correspondencias correctas entre imágenes.